In [14]:
# -*- coding: utf-8 -*-
"""
Created on Sun Aug 16 17:10:53 2020

@author: wanxiang.shen@u.nus.edu
"""

import warnings, os
warnings.filterwarnings("ignore")


import pandas as pd
import numpy as np
from glob import glob
from joblib import load, dump
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_auc_score
from sklearn.metrics import auc as calculate_auc

import matplotlib.pyplot as plt
import seaborn as sns

from aggmap import AggMap, AggModel, loadmap
from aggmap import show
np.random.seed(666) #just for reaptable results



In [2]:
flist = glob('../data/*.csv.gzip')
flist = pd.Series(flist).sort_values().tolist()
fall = []
for i in flist:
    df1 = pd.read_csv(i, compression='gzip', index_col = 0)
    df1['class'] = i.split('/')[-1].split('.csv')[0]
    fall.append(df1)
    
df = pd.concat(fall, axis=0)

In [3]:
dfx = df[df.columns[:-1]]
dfy = df[df.columns[-1:]]
dfx = np.log2(dfx + 1) #apply log2(x + 1)

In [4]:
data_save_folder = '/raid/shenwanxiang/transcriptome/pan-cancer'
mp = loadmap('/raid/shenwanxiang/agg_mp_object/pan-cancer.mp')


X_ns = load(os.path.join(data_save_folder, 'RP1_noisys.data')) 
X = X_ns[0]
Y = pd.get_dummies(dfy['class']).values

In [5]:
mp.fmap_shape

(102, 102)

In [6]:
dfy.shape, dfx.shape, X.shape, Y.shape

((10446, 1), (10446, 10381), (10446, 102, 102, 1), (10446, 33))

In [7]:
class_ = pd.get_dummies(dfy['class']).columns
class_

Index(['01_ACC', '02_BLCA', '03_BRCA', '04_CESC', '05_CHOL', '06_COAD',
       '07_DLBC', '08_ESCA', '09_GBM', '10_HNSC', '11_KICH', '12_KIRC',
       '13_KIRP', '14_LAML', '15_LGG', '16_LIHC', '17_LUAD', '18_LUSC',
       '19_MESO', '20_OV', '21_PAAD', '22_PCPG', '23_PRAD', '24_READ',
       '25_SARC', '26_SKCM', '27_STAD', '28_TGCT', '29_THCA', '30_THYM',
       '31_UCEC', '32_UCS', '33_UVM'],
      dtype='object')

In [8]:
dfy['idx'] = range(len(dfy))

In [9]:
dfy

,class,idx
TCGA-OR-A5J1-01A-11R-A29S-07,01_ACC,0
TCGA-OR-A5J2-01A-11R-A29S-07,01_ACC,1
TCGA-OR-A5J3-01A-11R-A29S-07,01_ACC,2
TCGA-OR-A5J5-01A-11R-A29S-07,01_ACC,3
TCGA-OR-A5J6-01A-31R-A29S-07,01_ACC,4
...,...,...
TCGA-YZ-A980-01A-11R-A405-07,33_UVM,10441
TCGA-YZ-A982-01A-11R-A405-07,33_UVM,10442
TCGA-YZ-A983-01A-11R-A405-07,33_UVM,10443
TCGA-YZ-A984-01A-11R-A405-07,33_UVM,10444


## 10 fold cv performances 

In [ ]:
outer_fold = 10

each_fold_results = []
outer = StratifiedKFold(n_splits = outer_fold, shuffle = True)
outer_idx = outer.split(X, dfy.values)

print('#'*50  )
run_one_res = []
for i in range(10):

    fold_num = "fold_%s" % str(i).zfill(2) 
    save_path = './fold_results_c1/%s' % fold_num
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    old_save_path = '../fold_results_c1/%s' % fold_num
    test_index = pd.read_csv(os.path.join(old_save_path, 'test_true_label.csv'), index_col=0).index
    test_idx = dfy[dfy.index.isin(test_index)]['idx'].tolist()
    train_idx = dfy[~dfy.index.isin(test_index)]['idx'].tolist()


    testY = Y[test_idx]
    testX = X[test_idx]

    trainX = X[train_idx]
    trainY = Y[train_idx]

    test_true_label = pd.DataFrame(testY).idxmax(axis=1).to_frame(name = 'y_true')
    test_true_label.index = dfy.iloc[test_idx].index
    test_true_label = test_true_label.join(dfy.iloc[test_idx])
    test_true_label.to_csv(os.path.join(save_path, 'test_true_label.csv'))


    print("\n input train and test X shape is %s, %s " % (trainX.shape,  testX.shape))

    clf = AggModel.MultiClassEstimator(epochs = 100, batch_size = 64, lr = 1e-3,  
                                       gpuid = 6, verbose = 1, metric = 'ACC', ) #
    clf.fit(trainX, trainY) 

    clf._model.save(os.path.join(save_path, 'model.h5'))
    
    test_pred_label = pd.DataFrame(clf.predict(testX)).idxmax(axis=1).to_frame(name = 'y_pred')
    test_pred_label.index = test_true_label.index
    test_pred_label.to_csv(os.path.join(save_path, 'test_pred_label.csv'))

    test_avg_loss, test_avg_acc = clf._model.evaluate(testX, testY, verbose=0)
    print('test_avg_loss: %.3f, test_avg_acc: %.3f, ' % (test_avg_loss, test_avg_acc))

    each_fold_results.append(test_avg_acc)

##################################################

 input train and test X shape is (9401, 102, 102, 1), (1045, 102, 102, 1) 
{'epochs': 100, 'lr': 0.001, 'conv1_kernel_size': 13, 'dense_layers': [128], 'dense_avf': 'relu', 'batch_size': 64, 'dropout': 0.0, 'batch_norm': False, 'n_inception': 2, 'monitor': 'val_loss', 'patience': 10000, 'random_state': 32, 'verbose': 1, 'name': 'AggMap MultiClass Estimator', 'gpuid': '6'}
Train on 9401 samples, validate on 9401 samples
Epoch 1/100
9401/9401 [==============================] - 12s 1ms/sample - loss: 3.0286 - accuracy: 0.1686 - val_loss: 2.0385 - val_accuracy: 0.4541
Epoch 2/100
9401/9401 [==============================] - 8s 863us/sample - loss: 1.2060 - accuracy: 0.6620 - val_loss: 0.6603 - val_accuracy: 0.8214
Epoch 3/100
9401/9401 [==============================] - 8s 873us/sample - loss: 0.5496 - accuracy: 0.8423 - val_loss: 0.4192 - val_accuracy: 0.8772
Epoch 4/100
9401/9401 [==============================] - 8s 882us/sample - loss

In [12]:
pd.DataFrame(each_fold_results)

,0
0,0.954067
1,0.950239
2,0.943541
3,0.949282
4,0.951196
5,0.955024
6,0.950192
7,0.950192
8,0.947318
9,0.961686


In [13]:
pd.DataFrame(each_fold_results).mean()

0    0.951274
dtype: float64